<a href="https://colab.research.google.com/github/gabrielfernandorey/EDVAI/blob/main/PySpark/PySpark_02.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# PySpark - Ejercicios clase 8

### Instalación y carga de Pyspark

In [1]:
!pip install pyspark

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 310.8/310.8 MB 3.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for pyspark: filename=pyspark-3.4.0-py2.py3-none-any.whl size=311317145 sha256=f934f92639664563b06b8eca3ecd6fa33f665da9fcf9ccc51d70d94d1463bb71
  Stored in directory: /root/.cache/pip/wheels/7b/1b/4b/3363a1d04368e7ff0d408e57ff57966fcdf00583774e761327
Successfully built pyspark


In [2]:
from pyspark.sql import SparkSession

In [3]:
spark = SparkSession.builder.appName('test_pyspark').getOrCreate()

### Librerías necesarias

In [4]:
from pyspark.sql.types import StringType, BooleanType, FloatType, IntegerType, DoubleType, DateType
import pyspark.sql.functions as F
from pyspark.sql.functions import sum, col, desc, asc, count, countDistinct, round, max, min, avg
from pyspark.sql.functions import to_timestamp,date_format
from pyspark.sql.window import Window

from pyspark.ml import Transformer
from pyspark.ml.param.shared import HasInputCol, HasOutputCol, HasInputCols, HasOutputCols, Param, Params, TypeConverters
from pyspark import keyword_only
from pyspark.ml import Pipeline, PipelineModel
from pyspark.ml import Model
from pyspark.ml import Estimator

from datetime import datetime
import numpy as np

### Importamos datos

In [5]:
!wget https://data-engineer-edvai.s3.amazonaws.com/f1/results.csv

--2023-05-06 20:31:07--  https://data-engineer-edvai.s3.amazonaws.com/f1/results.csv
Resolving data-engineer-edvai.s3.amazonaws.com (data-engineer-edvai.s3.amazonaws.com)... 52.216.76.28, 3.5.8.196, 52.217.142.57, ...
Connecting to data-engineer-edvai.s3.amazonaws.com (data-engineer-edvai.s3.amazonaws.com)|52.216.76.28|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1633624 (1.6M) [text/csv]
Saving to: ‘results.csv’

results.csv         100%[===================>]   1.56M  1.42MB/s    in 1.1s    

2023-05-06 20:31:09 (1.42 MB/s) - ‘results.csv’ saved [1633624/1633624]



In [6]:
!wget https://data-engineer-edvai.s3.amazonaws.com/f1/drivers.csv

--2023-05-06 20:31:09--  https://data-engineer-edvai.s3.amazonaws.com/f1/drivers.csv
Resolving data-engineer-edvai.s3.amazonaws.com (data-engineer-edvai.s3.amazonaws.com)... 52.216.76.28, 3.5.8.196, 52.217.142.57, ...
Connecting to data-engineer-edvai.s3.amazonaws.com (data-engineer-edvai.s3.amazonaws.com)|52.216.76.28|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 93568 (91K) [text/csv]
Saving to: ‘drivers.csv’

drivers.csv         100%[===================>]  91.38K   249KB/s    in 0.4s    

2023-05-06 20:31:11 (249 KB/s) - ‘drivers.csv’ saved [93568/93568]



In [7]:
!wget https://data-engineer-edvai.s3.amazonaws.com/f1/constructors.csv

--2023-05-06 20:31:11--  https://data-engineer-edvai.s3.amazonaws.com/f1/constructors.csv
Resolving data-engineer-edvai.s3.amazonaws.com (data-engineer-edvai.s3.amazonaws.com)... 52.216.76.28, 3.5.8.196, 52.217.142.57, ...
Connecting to data-engineer-edvai.s3.amazonaws.com (data-engineer-edvai.s3.amazonaws.com)|52.216.76.28|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 17384 (17K) [text/csv]
Saving to: ‘constructors.csv’

constructors.csv    100%[===================>]  16.98K  --.-KB/s    in 0s      

2023-05-06 20:31:11 (38.6 MB/s) - ‘constructors.csv’ saved [17384/17384]



In [ ]:
# Con esta alternativa algunos tipos de datos no se importan de forma adecuada
# df = spark.read.option("header","true").csv("*.csv")

In [8]:
# Con esta alternativa los tipos de datos se importan de forma adecuada (siempre verificar!)
df_results = spark.read.option("header","true").option("inferSchema", "true") .csv("results.csv")

In [9]:
df_drivers = spark.read.option("header","true").option("inferSchema", "true") .csv("drivers.csv")

In [10]:
df_constructors = spark.read.option("header","true").option("inferSchema", "true") .csv("constructors.csv")

### Visualizamos schema

In [11]:
df_results.printSchema()

root
 |-- resultId: integer (nullable = true)
 |-- raceId: integer (nullable = true)
 |-- driverId: integer (nullable = true)
 |-- constructorId: integer (nullable = true)
 |-- number: string (nullable = true)
 |-- grid: integer (nullable = true)
 |-- position: string (nullable = true)
 |-- positionText: string (nullable = true)
 |-- positionOrder: integer (nullable = true)
 |-- points: double (nullable = true)
 |-- laps: integer (nullable = true)
 |-- time: string (nullable = true)
 |-- milliseconds: string (nullable = true)
 |-- fastestLap: string (nullable = true)
 |-- rank: string (nullable = true)
 |-- fastestLapTime: string (nullable = true)
 |-- fastestLapSpeed: string (nullable = true)
 |-- statusId: integer (nullable = true)



In [12]:
df_results.show(5)

+--------+------+--------+-------------+------+----+--------+------------+-------------+------+----+-----------+------------+----------+----+--------------+---------------+--------+
|resultId|raceId|driverId|constructorId|number|grid|position|positionText|positionOrder|points|laps|       time|milliseconds|fastestLap|rank|fastestLapTime|fastestLapSpeed|statusId|
+--------+------+--------+-------------+------+----+--------+------------+-------------+------+----+-----------+------------+----------+----+--------------+---------------+--------+
|       1|    18|       1|            1|    22|   1|       1|           1|            1|  10.0|  58|1:34:50.616|     5690616|        39|   2|      1:27.452|        218.300|       1|
|       2|    18|       2|            2|     3|   5|       2|           2|            2|   8.0|  58|     +5.478|     5696094|        41|   3|      1:27.739|        217.586|       1|
|       3|    18|       3|            3|     7|   7|       3|           3|            3|  

In [13]:
df_drivers.printSchema()

root
 |-- driverId: integer (nullable = true)
 |-- driverRef: string (nullable = true)
 |-- number: string (nullable = true)
 |-- code: string (nullable = true)
 |-- forename: string (nullable = true)
 |-- surname: string (nullable = true)
 |-- dob: date (nullable = true)
 |-- nationality: string (nullable = true)
 |-- url: string (nullable = true)



In [14]:
df_drivers.show(5)

+--------+----------+------+----+--------+----------+----------+-----------+--------------------+
|driverId| driverRef|number|code|forename|   surname|       dob|nationality|                 url|
+--------+----------+------+----+--------+----------+----------+-----------+--------------------+
|       1|  hamilton|    44| HAM|   Lewis|  Hamilton|1985-01-07|    British|http://en.wikiped...|
|       2|  heidfeld|    \N| HEI|    Nick|  Heidfeld|1977-05-10|     German|http://en.wikiped...|
|       3|   rosberg|     6| ROS|    Nico|   Rosberg|1985-06-27|     German|http://en.wikiped...|
|       4|    alonso|    14| ALO|Fernando|    Alonso|1981-07-29|    Spanish|http://en.wikiped...|
|       5|kovalainen|    \N| KOV|  Heikki|Kovalainen|1981-10-19|    Finnish|http://en.wikiped...|
+--------+----------+------+----+--------+----------+----------+-----------+--------------------+
only showing top 5 rows



In [15]:
df_constructors.printSchema()

root
 |-- constructorId: integer (nullable = true)
 |-- constructorRef: string (nullable = true)
 |-- name: string (nullable = true)
 |-- nationality: string (nullable = true)
 |-- url: string (nullable = true)



In [16]:
df_constructors.show(5)

+-------------+--------------+----------+-----------+--------------------+
|constructorId|constructorRef|      name|nationality|                 url|
+-------------+--------------+----------+-----------+--------------------+
|            1|       mclaren|   McLaren|    British|http://en.wikiped...|
|            2|    bmw_sauber|BMW Sauber|     German|http://en.wikiped...|
|            3|      williams|  Williams|    British|http://en.wikiped...|
|            4|       renault|   Renault|     French|http://en.wikiped...|
|            5|    toro_rosso|Toro Rosso|    Italian|http://en.wikiped...|
+-------------+--------------+----------+-----------+--------------------+
only showing top 5 rows



### Insertar en la tabla driver_results los corredores con mayor cantidad de puntos en la historia.

In [17]:
# Creamos vista
df_results.createOrReplaceTempView("vista_results")
df_drivers.createOrReplaceTempView("vista_drivers")

In [49]:
query = """SELECT vista_results.driverId, vista_drivers.driverRef, SUM(vista_results.points) as total
           FROM vista_results 
           JOIN vista_drivers ON vista_results.driverId = vista_drivers.driverId
           GROUP BY vista_results.driverId, vista_drivers.driverRef
           ORDER BY total DESC """

df_pilotos = spark.sql(query)

In [50]:
df_pilotos.show()

+--------+------------------+------+
|driverId|         driverRef| total|
+--------+------------------+------+
|       1|          hamilton|4308.5|
|      20|            vettel|3077.0|
|       4|            alonso|2021.0|
|       8|         raikkonen|1873.0|
|     830|    max_verstappen|1792.5|
|     822|            bottas|1775.0|
|       3|           rosberg|1594.5|
|      30|michael_schumacher|1566.0|
|     817|         ricciardo|1289.0|
|      18|            button|1235.0|
|      13|             massa|1167.0|
|     815|             perez|1059.0|
|      17|            webber|1047.5|
|     117|             prost| 798.5|
|     844|           leclerc| 724.0|
|     832|             sainz| 680.5|
|      22|       barrichello| 658.0|
|     102|             senna| 614.0|
|      14|         coulthard| 535.0|
|     807|        hulkenberg| 521.0|
+--------+------------------+------+
only showing top 20 rows



### Insertar en la tabla constructor_result quienes obtuvieron más puntos en el Spanish Grand Prix en el año 1991

In [57]:
# Creamos vista
df_constructors.createOrReplaceTempView("vista_team")

In [58]:
query = """SELECT vista_results.constructorId, vista_team.constructorRef, SUM(vista_results.points) as total
           FROM vista_results 
           JOIN vista_team ON vista_results.constructorId = vista_team.constructorId
           GROUP BY vista_results.constructorId, vista_team.constructorRef
           ORDER BY total DESC """

df_SGP = spark.sql(query)

In [61]:
df_SGP.show(30)

+-------------+--------------+-----------------+
|constructorId|constructorRef|            total|
+-------------+--------------+-----------------+
|            6|       ferrari|          9924.27|
|          131|      mercedes|6726.639999999999|
|            1|       mclaren|           6085.5|
|            9|      red_bull|           6027.0|
|            3|      williams|           3593.0|
|            4|       renault|           1777.0|
|           10|   force_india|           1098.0|
|           32|    team_lotus|            995.0|
|           22|      benetton|            861.5|
|           25|       tyrrell|            711.0|
|          208|      lotus_f1|            706.0|
|           34|       brabham|            631.0|
|           15|        sauber|            557.0|
|           66|           brm|            537.5|
|            5|    toro_rosso|            500.0|
|           27|        ligier|            388.0|
|           51|          alfa|            341.0|
|          170| coop